In [ ]:
import matplotlib.pyplot as plt
import numpy as np
#Function that checks how many iterations it takes for a number c to diverge
#If function returns a number n<50 then c diverges
#If function returns n = 50 then the number c converges
def mandelbrotConverge(c):
    #Initialize variables
    maxIterations = 50
    currentIteration = 0
    z = 0
    
    #While absolute value of z < 2 and currentIterations < 50
    while (abs(z) < 2) and (currentIteration < maxIterations):
        #z = z^2 + c
        z = (z**2) + c
        
        #Add one to current iterations
        currentIteration+=1

    #Return number of iterations
    return currentIteration

#Construct 1000x1000 grid
x, y = np.mgrid[-2:1:1000j , -1.5:1.5:1000j]
c = x + 1j * y

#For each point in grid, check how many iterations a point takes until it diverges
for i in range(1000):
    for k in range(1000):
       c[i][k] = mandelbrotConverge(c[i][k])

#Remove the imaginary part of each value in the grid      
c = c.real

#Plot and produce the image
plt.imshow(c,cmap='magma',vmin=1, vmax=50)
plt.axis('off')
plt.show()

In [ ]:
import numpy as np


############################
## Task 1
############################

matrix_p = np.random.rand(5,5) # random 5x5 matrix

for i in range(5):
    # normalize each row
    row_sum = np.sum(matrix_p[i]) 
    if row_sum != 0:
        matrix_p[i] = matrix_p[i] / row_sum
    else:
        matrix_p[i] = [0.2, 0.2, 0.2, 0.2, 0.2]  # if row sum is 0, assign equal probabilities

print("-----Task 1-----")
print(matrix_p)

############################
## Task 2
############################

vector_p = np.random.rand(5)
# normalize the vector
if np.sum(vector_p) != 0:
    vector_p = vector_p / np.sum(vector_p)
else:
    vector_p = np.array([0.2, 0.2, 0.2, 0.2, 0.2])  # if sum is 0, assign equal probabilities

p_50 = vector_p.copy() # copy the values

for i in range(50):
    p_50 = np.dot(matrix_p.T, p_50) # apply the transition rule

print("-----Task 2-----")
print(p_50)

############################
## Task 3
############################

eigenvalues, eigenvectors = np.linalg.eig(matrix_p.T) # get eigenvalues and eigenvectors of the transposed matrix

i = (abs(eigenvalues.real-1)<1e-4) # find index of eigenvalue close to 1 (computational tolerance)
v = eigenvectors[:, i].real #  eigenvector corresponding to eigenvalue 1

station_dist = v/np.sum(v) # normalize the eigenvector; Scale the eigenvector so that its entries sum to 1

print("-----Task 3-----")
print(station_dist) # stationary distribution


############################
## Task 4
############################

components_diff = p_50 - station_dist.T # component-wise difference between p50 and the stationary distribution
match = np.all(np.abs(components_diff) < 1e-5) # check if all components are within 10^(-5)

print("-----Task 4-----")

print(components_diff)
if match: 
    print("They match well.")
else:
    print("They do not match well.")

In [ ]:
import numpy as np

def row_normalize(M):
    row_sums = M.sum(axis=1, keepdims=True)
    # if a row is all zeros, make it uniform
    row_sums[row_sums == 0.0] = 1.0
    return M / row_sums

def random_stochastic_matrix(n=5, seed=0):
    rng = np.random.default_rng(seed)
    P = rng.random((n, n))
    return row_normalize(P)

def random_probability_vector(n=5, seed=1):
    rng = np.random.default_rng(seed)
    v = rng.random(n)
    return v / v.sum()

def apply_transitions(P, p0, steps=50):
    PT = P.T
    p = p0.copy()
    for _ in range(steps):
        p = PT @ p
        p = p / p.sum()  
    return p

def stationary_distribution(P, atol=1e-12):
    # Eigenvector of P^T for eigenvalue ~1
    w, V = np.linalg.eig(P.T)
    # Find eigenvalue closest to 1
    idx = np.argmin(np.abs(w - 1.0))
    v = np.real(V[:, idx])
    # flip sign if needed
    if v.sum() < 0:
        v = -v
    # Normalize to sum 1
    v = v / v.sum()
    # Small negatives to zero due to numerical noise
    v[np.abs(v) < atol] = 0.0
    return v

def main():
    n = 5
    P = random_stochastic_matrix(n=n, seed=42)
    p0 = random_probability_vector(n=n, seed=123)

    p50 = apply_transitions(P, p0, steps=50)
    v = stationary_distribution(P)

    diff = np.abs(p50 - v)
    max_diff = diff.max()

    print("P:\n", P)
    print("Initial p0:", p0)
    print("p after 50 steps:", p50)
    print("Stationary distribution v:", v)
    print("Component-wise abs difference:", diff)
    print("Max difference:", max_diff)
    print("Match within 1e-5? ->", max_diff <= 1e-5)

if __name__ == "__main__":
    main()


In [ ]:
import numpy as np

# Create a random matrix P and normalize the rows
P = np.random.rand(5, 5)
P = P / P.sum(axis=1, keepdims=True)

# Create a random vector p and normalize it
p = np.random.rand(5)
p = p / p.sum()

# Apply the transition rule 50 times
for _ in range(50):
    p = np.dot(P.T, p)

# Find the eigenvalues and eigenvectors of p50
eigenvalues, eigenvectors = np.linalg.eig(P.T)

# Find the eigenvector corresponding to eigenvalue 1 and normalize it
idx = np.argmin(np.abs(eigenvalues - 1))
v = eigenvectors[:, idx]
v = v / v.sum()

# Check if the difference between p50 and v is less than 1-e5 component-wise
diff = (np.abs(p - v))
print(np.all(diff < 1e-5))

In [ ]:
from sympy import symbols, diff
import numpy as np
import math
import matplotlib.pyplot as plt
import sympy

def checkAnalytical(func,start,end,degree,fixed_c,numberPoints):
  x = symbols('x')
  #defining the taylor expansion
  x_range = np.linspace(start,end,numberPoints)
  taylor = np.zeros(numberPoints)
  for k in range(len(taylor)):
    for i in range(degree):
      a = ((x_range[k] - fixed_c) ** i) / math.factorial(i)
      taylor[k] = taylor[k] + a * func.diff(x,i).subs(x, fixed_c).evalf()
      #had to put evalf so it would be a number I could plot
  return taylor

#plotting the results with the test function
x = symbols('x')
f = x*sympy.sin(x) ** 2 + sympy.cos(x)
xVals = np.linspace(-10,10,100)
yVals = checkAnalytical(f,-10,10,100,0,100)
plt.plot(xVals,yVals, 'o', label = "Taylor")
yActuals = [f.subs(x,val) for val in xVals]
plt.plot(xVals,yActuals, label = "Actual function")
plt.legend()
plt.xlabel("x")
plt.ylabel("f(x)=xsin^2(x)+cos(x)")
plt.show()

In [ ]:
import pandas as pd
import time
from sympy import symbols, diff
import numpy as np
import math
import matplotlib.pyplot as plt
import sympy
def taylorFactorial(func,start,end,fixed_c,numberPoints,initial_degree,final_degree,degree_step):
  x = symbols('x')
  #setting up rows for my dataframe
  rows = int((final_degree - initial_degree)/degree_step)
  df = pd.DataFrame({'cutoff value': np.zeros(rows), 'difference': np.zeros(rows), 'time': np.zeros(rows)})
  xVals = np.linspace(start,end,numberPoints)
  #running the approx to different degrees
  for i in range(rows):
    #timing run time
    start_time = time.time()
    deg = initial_degree + i*degree_step
    df.loc[i, 'cutoff value'] = deg
    yVals = checkAnalytical(func,start,end,deg,fixed_c,numberPoints)
    sum = 0
    for k in range(numberPoints):
      sum = sum + abs(yVals[k] - func.subs(x, xVals[k]).evalf())
    df.loc[i, 'difference'] = sum
    end = time.time()
    df.loc[i, 'time'] = end - start_time
    #writing the csv file
  df.to_csv('taylor.csv')

x = symbols('x')
f = x*sympy.sin(x) ** 2 + sympy.cos(x)
taylorFactorial(f,-10,10,0,100,50,100,10)

In [ ]:
import numpy as np

def normalize_rows(P: np.ndarray) -> np.ndarray:
    row_sums = P.sum(axis=1, keepdims=True)
    row_sums[row_sums == 0] = 1.0
    return P / row_sums

def normalize_vec(p: np.ndarray) -> np.ndarray:
    s = p.sum()
    if s == 0:
        return np.ones_like(p) / p.size
    return p / s

def transition(P: np.ndarray, p: np.ndarray) -> np.ndarray:
    return P.T @ p

def stationary_distribution(P: np.ndarray) -> np.ndarray:
    w, V = np.linalg.eig(P.T)
    idx = np.argmin(np.abs(w - 1.0))
    vec = np.real(V[:, idx])
    if np.any(vec < 0):
        vec = np.abs(vec)
    vec = normalize_vec(vec)
    return vec



def main(seed: int = 0, steps: int = 50, n: int = 5, tol: float = 1e-5):
    rng = np.random.default_rng(seed)
    P = normalize_rows(rng.random((n, n)))
    p0 = normalize_vec(rng.random(n))
    p = p0.copy()
    for _ in range(steps):
        p = transition(P, p)
    p50 = p
    v = stationary_distribution(P)
    diff = np.abs(p50 - v)
    max_diff = diff.max()
    match = bool(max_diff <= tol)

    np.set_printoptions(precision=6, suppress=True)
    print("Transition matrix P (rows sum to 1):\n", P, "\n")
    print("Initial distribution p0:\n", p0, "\n")
    print(f"Distribution after {steps} transitions, p{steps}:\n", p50, "\n")
    print("Stationary distribution v:\n", v, "\n")
    print("|p50 - v|:\n", diff, "\n")
    print(f"max(|p50 - v|) = {max_diff:.8e}")
    print(f"Match within 1e-5? {match}")

if __name__ == '__main__':
    import sys
    seed = int(sys.argv[1]) if len(sys.argv) > 1 else 0
    main(seed=seed)


Transition matrix P (rows sum to 1):
 [[0.358343 0.151777 0.023051 0.009298 0.457531]
 [0.244865 0.162742 0.195702 0.145838 0.250852]
 [0.334471 0.001123 0.351505 0.013769 0.299133]
 [0.076283 0.374856 0.235142 0.130157 0.183562]
 [0.013577 0.059585 0.321519 0.310283 0.295035]] 

Initial distribution p0:
 [0.10376  0.269682 0.265254 0.185396 0.175908] 

Distribution after 50 transitions, p50:
 [0.194933 0.118167 0.244093 0.135411 0.307395] 

Stationary distribution v:
 [0.194933 0.118167 0.244093 0.135411 0.307395] 

|p50 - v|:
 [0. 0. 0. 0. 0.] 

max(|p50 - v|) = 4.99600361e-16
Match within 1e-5? True
